In [1]:
"""
NFT Collection Recommender System with Gradio UI and LLM Integration
- Vector search using multilingual-e5-large-instruct
- Top-k results passed to LLM (Phi-3.5-mini-instruct) for reasoning
"""

import pandas as pd
import numpy as np
import faiss
import gradio as gr
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

# Load NFT data
df = pd.read_csv("/Users/cpysleeper/comp631_proj/NFT_collections.csv")
documents = (df["title"].fillna("") + " " + df["text"].fillna("")).tolist()

# Load embedding model
embed_model = SentenceTransformer("intfloat/multilingual-e5-large-instruct")
doc_embeddings = embed_model.encode(
    [f"passage: {doc}" for doc in documents],
    convert_to_numpy=True,
    show_progress_bar=True
)

# Build FAISS index
dim = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(doc_embeddings)







Batches:   0%|          | 0/1240 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3.5-mini-instruct")
phi_model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3.5-mini-instruct")
llm = pipeline("text-generation", model=phi_model, tokenizer=tokenizer, torch_dtype=torch.float16, device_map="auto")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use mps:0


In [3]:
# Define the query interface
def search_nft_with_reasoning(query: str, k: int = 3) -> str:
    query_embed = embed_model.encode(f"query: {query}", convert_to_numpy=True)
    query_embed = np.expand_dims(query_embed, axis=0)

    _, indices = index.search(query_embed, k)
    top_docs = df.iloc[indices[0]]

    context = "\n---\n".join(
        f"Title: {row['title']}\nDescription: {row['text']}" for _, row in top_docs.iterrows()
    )

    prompt = (
        f"You are an expert NFT advisor. Based on the following NFT collections,\n"
        f"recommend which are the most relevant for the user query: '{query}'.\n"
        f"\nNFTs:\n{context}\n\nAnswer:"
    )

    response = llm(prompt, max_new_tokens=256, do_sample=False)[0]['generated_text']
    return response[len(prompt):].strip()

# Gradio UI
demo = gr.Interface(
    fn=search_nft_with_reasoning,
    inputs=gr.Textbox(label="Enter your NFT interest or query"),
    outputs=gr.Textbox(label="Recommended Collections with Reasoning"),
    title="NFT Collection Recommender with LLM",
    description="This app retrieves top-k NFT collections using semantic search and explains the matches using a tiny Mistral model."
)

if __name__ == "__main__":
    demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


/Users/cpysleeper/miniconda3/envs/gdal0/lib/python3.9/site-packages/transformers/pytorch_utils.py:332: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_elements = torch.tensor(test_elements)
